In [1]:
import sqlite3
from pyteomics import mass as massC
import struct
import numpy as np
import pandas as pd

import keras.backend as K
from keras.layers.convolutional import Conv1D
from keras.layers.core import Dense, Dropout, Masking
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import load_model as keras_load_model
from keras.models import Sequential


Using TensorFlow backend.
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflo

In [2]:
def binarySearch (arr, l, r, x, tol):

    # Check base case
    if r >= l:
        mid = int(l + (r - l)/2)

        # If element is present at the middle itself
        if arr[mid][0] > x - tol and arr[mid][0] < x + tol:
            return mid

        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid][0] > x + tol:
            return binarySearch(arr, l, mid-1, x, tol)

        # Else the element can only be present in right subarray
        else:
            return binarySearch(arr, mid+1, r, x, tol)

    else:
        # Element is not present in the array
        return -1

In [3]:
#Healper Function, Converts bitstring to float
def convertFloat(element):
    return struct.iter_unpack('>f', element)

In [4]:
def lossConvert(loss, charge):
     if loss == '':
         return 0
     elif loss == 'n':
         return massC.calculate_mass(formula='NH3', charge = charge)
     elif loss == 'o':
         return massC.calculate_mass(formula='H2O', charge = charge)

def getIonMasses(peptide, types=('b', 'y'), maxcharge=2):
    """
    The function generates all possible m/z for fragments of types
    `types` and of charges from 1 to `maxharge`.
    """
    ions = {"b1": [], "b2": [], "bn1": [], "bn2": [], "bo1": [], "bo2": [], "y1": [], "y2": [], "yn1": [], "yn2": [], "yo1": [], "yo2": [], }
    losses = ['', 'n', 'o']
    for ion_type in types:
        for charge in range(1, maxcharge+1):
            for lossT in losses:
                key = ion_type  + lossT + str(charge)
                loss = lossConvert(lossT, charge)
                for i in range(1, len(peptide)-1):
                    if ion_type[0] in 'abc':
                        ions[key].append( massC.fast_mass(peptide[:i], ion_type=ion_type, charge=charge))
                    else:
                        ions[key].append( massC.fast_mass(peptide[i:], ion_type=ion_type, charge=charge))
    return ions


In [5]:
#Connect to database
conn = sqlite3.connect('testLibDuplicateSpectra.db')
xtrainc2 = []
xtrainc3 = []
ytrainc2 = []
ytrainc3 = []

c2Arr = []
c3Arr = []

#peptideTemp = {"peptide":"", "modification": 'null', "ions":{}}
peptideTemp = {"peptide":"", "ions":{}}

peptideCnt = 0
specCnt = 0

print ("Pulling Data from Database")
c = conn.cursor()
c.execute("SELECT * FROM PeptideTable")
pepTable = c.fetchall()

cnt = 0

#Iterate through table and pull information about each peptide and its scans
#for ind in pepTable:
for j in range(15):

    #Match peptide in table to peptide in Spectra Table
    #pepID = (str(ind[0]), )
    pepID = (str(pepTable[j][0]), )

    #peptide = ind[1].split('.')[1]
    peptide = pepTable[j][1].split('.')[1]

    if '(' in peptide or 'Z' in peptide or 'B' in peptide or 'U' in peptide:
        continue

    #charge = int(ind[4])
    charge = pepTable[j][4]

    #print(pepID, ":" ,peptide)

    peptideCnt += 1;
    c.execute('SELECT *,rowid FROM SpectraTable WHERE peptideID=?', pepID)
    spectrums = c.fetchall()

    #Make sure peptide has more than 50 spectrums for accurate training.
    specCnt += 1 #increment peptide count

    #Search returned list of matched spectrums for each peptide
    for element in spectrums:
        ions = getIonMasses(peptide)

        spectrum = [] #Holder variable for spectrum

        #append the scan id
        #idList.append(element[4])
        #massList.append(float(element[3])/1000);
        #Grab mzArr and intArr from
        mzArr = list(convertFloat(element[1]))
        intArr = list(convertFloat(element[2]))

        for key, massList in ions.items():
            intensities = []
            for mass in massList:
                #print(key,":" ,mass)
                found = False
                mass = mass
                tolerance = 400 * mass/1000000
                #cdistance = 10000
                #cValue = 0

                result = binarySearch(mzArr, 0, len(mzArr) - 1, mass, tolerance)
                if result != -1:
                    intensities.append(intArr[result][0])
                else:
                    intensities.append(0)
            ions[key] = intensities


        tmp = peptideTemp
        tmp['peptide'] = peptide
        tmp['ions'] = ions
        #print(tmp)


        if charge == 2:
            #xtrainc2.append(peptide)
            #ytrainc2.append(ions)
            c2Arr.append(tmp)
        elif charge == 3:
            #xtrainc3.append(peptide)
            #ytrainc3.append(ions)
            c3Arr.append(tmp)
        else:
            cnt += 1
            #print("Charge:", charge, "invalid")
print("Done")

Pulling Data from Database
Done


In [6]:
df = pd.DataFrame(c2Arr)
df = df.sample(frac=1)
df.head()

,peptide,ions
482,YIREPEHPASFYEVLYFQDPQA,"{'b1': [0, 0, 82.9000015258789, 70.90000152587..."
640,YIREPEHPASFYEVLYFQDPQA,"{'b1': [0, 0, 82.9000015258789, 70.90000152587..."
526,YIREPEHPASFYEVLYFQDPQA,"{'b1': [0, 0, 82.9000015258789, 70.90000152587..."
81,YIREPEHPASFYEVLYFQDPQA,"{'b1': [0, 0, 82.9000015258789, 70.90000152587..."
176,YIREPEHPASFYEVLYFQDPQA,"{'b1': [0, 0, 82.9000015258789, 70.90000152587..."



#Encode Sequence

#read the matrix a csv file on github
nlf = pd.read_csv('https://raw.githubusercontent.com/dmnfarrell/epitopepredict/master/epitopepredict/mhcdata/NLF.csv',index_col=0)

def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def nlf_encode(seq):    
    x = pd.DataFrame([nlf[i] for i in seq]).reset_index(drop=True)  
    #show_matrix(x)
    e = x.values.flatten()
    return e

modPeptides = df['peptide'].copy()
for i, pep in enumerate(modPeptides):
    modPeptides.iloc[i] = nlf_encode(pep)

"""
for i in modPeptides:
    print(i.shape)
"""

modPeptides.head()
print(modPeptides.iloc[0])

In [7]:
import seaborn as sns
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
#Encode Sequence
def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    #show_matrix(a)
    e = a.values.flatten()
    return e

modPeptides = df['peptide'].copy()
for i, pep in enumerate(modPeptides):
    modPeptides.iloc[i] = one_hot_encode(pep)

"""
for i in modPeptides:
    print(i.shape)
"""

modPeptides.head()
print(modPeptides.iloc[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
df = pd.concat([df, modPeptides], axis = 1)
df.columns = ['sequence', 'ions', 'encodedSequence']
df = df[['sequence', 'encodedSequence', 'ions']]
df.head()
print(np.array(df['encodedSequence'].iloc[0]).shape)

(440,)


In [9]:
ionsProcessed = df['ions'].copy()
for i, ionDict in enumerate(ionsProcessed):
    container  = []
    for key,value in ionDict.items():
        ionList = []
        for j in value:
            ionList.append(j)
        container.append(np.array(ionList))
    ionsProcessed.iloc[i] = np.array(container)
    #print(np.array(container).shape)
#ionsProcessed.head()

df = pd.concat([df, ionsProcessed], axis = 1)
df.columns = ['sequence', 'encodedSequence', 'ions', 'ionsProcessed']
df.head()

,sequence,encodedSequence,ions,ionsProcessed
482,YIREPEHPASFYEVLYFQDPQA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
640,YIREPEHPASFYEVLYFQDPQA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
526,YIREPEHPASFYEVLYFQDPQA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
81,YIREPEHPASFYEVLYFQDPQA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
176,YIREPEHPASFYEVLYFQDPQA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."


In [13]:
inputArr = np.array(df['encodedSequence'])
inputArr = np.array([np.array(i) for i in inputArr])
outputArr = np.array([np.array(i) for i in df['ionsProcessed']])

inputArr = np.reshape(inputArr, (inputArr.shape[0], 1, inputArr.shape[1]))
outputArr = np.reshape(outputArr, (outputArr.shape[0], 240))

xTrain = inputArr[:650]
xTest = inputArr[650:]

yTrain = outputArr[:650]
yTest = outputArr[650:]
print(inputArr.shape)
print(outputArr.shape)


(740, 1, 440)
(740, 240)


In [14]:
model = Sequential()

model.add(Bidirectional(LSTM(3, input_shape = xTrain.shape)))
#model.add(Dense(20))
#model.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(240, activation='relu')))
model.add(Dense(240, activation='relu'))

model.compile(
    loss="mean_squared_error",
    optimizer="adam")
model.fit(xTrain, yTrain, epochs = 100)


Epoch 1/100
650/650 [==============================] - 1s 1ms/step - loss: 1965397.9665
Epoch 2/100
650/650 [==============================] - 0s 124us/step - loss: 1965337.6119
Epoch 3/100
650/650 [==============================] - 0s 137us/step - loss: 1965238.8065
Epoch 4/100
650/650 [==============================] - 0s 150us/step - loss: 1965120.2612
Epoch 5/100
650/650 [==============================] - 0s 137us/step - loss: 1965002.7823
Epoch 6/100
650/650 [==============================] - 0s 120us/step - loss: 1964892.5715
Epoch 7/100
650/650 [==============================] - 0s 113us/step - loss: 1964789.5377
Epoch 8/100
650/650 [==============================] - 0s 150us/step - loss: 1964691.8962
Epoch 9/100
650/650 [==============================] - 0s 140us/step - loss: 1964598.3350
Epoch 10/100
650/650 [==============================] - 0s 194us/step - loss: 1964507.7908
Epoch 11/100
650/650 [==============================] - 0s 173us/step - loss: 1964419.4642
Epoch 12/

650/650 [==============================] - 0s 168us/step - loss: 1958579.3362
Epoch 89/100
650/650 [==============================] - 0s 195us/step - loss: 1958505.8731
Epoch 90/100
650/650 [==============================] - 0s 183us/step - loss: 1958432.5454
Epoch 91/100
650/650 [==============================] - 0s 198us/step - loss: 1958359.2192
Epoch 92/100
650/650 [==============================] - 0s 146us/step - loss: 1958285.9223
Epoch 93/100
650/650 [==============================] - 0s 135us/step - loss: 1958212.6335
Epoch 94/100
650/650 [==============================] - 0s 221us/step - loss: 1958139.3854
Epoch 95/100
650/650 [==============================] - 0s 178us/step - loss: 1958066.1577
Epoch 96/100
650/650 [==============================] - 0s 122us/step - loss: 1957992.9162
Epoch 97/100
650/650 [==============================] - 0s 134us/step - loss: 1957919.7173
Epoch 98/100
650/650 [==============================] - 0s 131us/step - loss: 1957846.4735
Epoch 99/100

In [15]:
predictions = model.predict(xTest)
for i in predictions:
    print(i)

[0.00000000e+00 0.00000000e+00 1.20713148e+01 1.22488155e+01
 0.00000000e+00 1.18046970e+01 0.00000000e+00 0.00000000e+00
 1.19925556e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.14781551e+01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.15554218e+01 1.20784788e+01
 0.00000000e+00 1.22657423e+01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.23340006e+01 1.21847401e+01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.21028624e+01 1.22987423e+01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11939564e+01
 0.00000000e+00 0.00000000e+00 1.12111149e+01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000